# Análise de Fatores de Risco para AVC
## 1. Introdução
- Breve descrição do problema e objetivos do projeto.
- Fonte dos dados (link e descrição).
- Tecnologias utilizadas.

## 2. Importação das Bibliotecas e do Dataset

## 3. Entendimento Inicial dos Dados
- shape, head(), info(), describe()
- Verificação de tipos de dados, valores únicos, nulos, etc.

## 4. Limpeza e Pré-processamento
- Tratamento de valores ausentes (ex: `bmi`)
- Conversão de tipos e renomeação de colunas
- Padronização de categorias
- Criação de novas variáveis, se necessário

## 5. Análise Exploratória (EDA)
### 5.1 Perfil demográfico e clínico
- Distribuição de idade, gênero, IMC, glicose
- Frequência de hipertensão, doenças cardíacas

### 5.2 Comparações com a variável alvo (stroke)
- Taxa de AVC por grupo etário, sexo, hipertensão etc.
- Gráficos de comparação (barplots, boxplots, etc.)

## 6. Agregações para Visualização no Power BI
- Exportar CSVs com:
  - AVC por idade/grupo etário
  - AVC por hipertensão, glicose, IMC (categorizado)
  - Tabelas de cruzamento por gênero, tipo de trabalho, estado civil

## 7. Conclusões e próximos passos
- Sumário de insights
- Oportunidades de visualização no Power BI

# 8. Salvando arquivos para o Power BI
- `to_csv()` dos dataframes agregados